In [ ]:
# ls /dev/tty.*

In [ ]:
# %pip install matplotlib
# %pip install ipywidgets
# %pip install telemetrix
# %pip install pyfirmata

In [ ]:
import numpy as np
import os
import cv2
from matplotlib import pyplot as plt
import pickle
import time
import ipywidgets as widgets
from IPython.display import display
import sys
from telemetrix import telemetrix

DIGITAL_B = 11 # the board LED
DIGITAL_G = 12
DIGITAL_R = 7
SERVO_PIN = 5
BUTTON_PIN = 10
# BUZZER = 13

WAIT_TIME = 2 # sleep time to keep forever loop open

# callback data indices
CB_PIN_MODE = 0
CB_PIN = 1
CB_VALUE = 2
CB_TIME = 3
flag = True

def the_callback(data):
    global flag
    value = data[CB_VALUE]
    if value == 0:
        print('System Reset')
        board.digital_write(DIGITAL_B, 1)
        board.digital_write(DIGITAL_R, 0)
        flag = False
    time.sleep(WAIT_TIME)

board = telemetrix.Telemetrix('/dev/tty.usbmodem1101', arduino_wait=6)
board.digital_write(DIGITAL_B, 1)

# taking a picture with your webcam
# board.digital_write(BUZZER, 0)

# first we create a camera object
camera = cv2.VideoCapture(0)

# now we take 2 pictures since the first pic tends to be too dark,
# as the code runs too fast for the exposure to be enough
for i in range(3):
    success, img_captured = camera.read()
    time.sleep(0.3) # waiting time for the webcam to get more light

cv2.imwrite('capturedPicture.png', img_captured) # saving the second pic we took
del camera # closes the access to the camera by closing the camera object

# Enter the paths for your image and cascade files here

imagePath = 'capturedPicture.png'
labelsPath = 'labels.pickle'

# Create the haar cascade (XML file that contains the data to detect faces)
faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml') # Face classifier

# create the recognizer
recognizer = cv2.face.LBPHFaceRecognizer_create()
# import the training data for the recognizer
recognizer.read("trainer.yml")

# import the lables created by the facesTrain program
label = {}
with open("labels.pickle", "rb") as pickleFile:
    original_labels = pickle.load(pickleFile)
    # invert the order:
    labels = {v:k for k, v in original_labels.items()}

# read the image
image = cv2.imread(imagePath)
image_RGB = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
plt.title("Original Image")
plt.imshow(image_RGB) # open the image as RGB
plt.xticks([]), plt.yticks([])
plt.show()

# transform the image to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
plt.title("Gray Image")
plt.imshow(gray, cmap='gray') # open the image as greyscale
plt.xticks([]), plt.yticks([])
plt.show()

# detect faces in the image
faces = faceCascade.detectMultiScale(
    gray,
    scaleFactor=1.21,
    minNeighbors=4,
    minSize=(20,20),
    flags = cv2.CASCADE_SCALE_IMAGE
)
print(faces)

# print "Found {0} faces!".format(len(faces))
print('Faces found: ', len(faces))

# Draw a rectangle around the faces
for (x, y, w, h) in faces:
    cv2.rectangle(image_RGB, (x, y), (x+w, y+h), (0, 255, 0), 5)
    roi_gray = gray[y:y+h, x:x+w]
    id_now, confidence = recognizer.predict(roi_gray)
    print("confidence = ", confidence)
    print("Face Recognized")
    print("Access Granted")
plt.title("Faces Found")
plt.imshow(image_RGB) # open the image as RGB
plt.xticks([]), plt.yticks([])
plt.show()

if confidence >= 70 and len(faces) > 0:
    board.digital_write(DIGITAL_B, 0)
    board.digital_write(DIGITAL_G, 1)
    board.set_pin_mode_servo(SERVO_PIN, 100, 3000)
    time.sleep(0.2)
    board.servo_write(SERVO_PIN, 180)
    time.sleep(1)
    board.servo_write(SERVO_PIN, 0)
    print("Opening the secret exit")
else:
    # board.digital_write(DIGITAL_B, 0)
    # board.digital_write(DIGITAL_R, 1)
    board.set_pin_mode_digital_input_pullup(BUTTON_PIN, callback = the_callback)
    print("Access Denied")
    print("Security Breach")
    while (flag):
        # board.digital_write(BUZZER, 1)
        board.digital_write(DIGITAL_R, 1)
        time.sleep(0.5)
        # board.digital_write(BUZZER, 0)
        board.digital_write(DIGITAL_R, 0)
        time.sleep(0.5)

board.shutdown()


In [ ]:
from telemetrix import telemetrix
import time

board = telemetrix.Telemetrix('/dev/tty.usbmodem1101', arduino_wait=6)

board.set_pin_mode_digital_output(13)

for i in range(5):
    board.digital_write(13, 1)
    time.sleep(0.5)
    board.digital_write(13, 0)
    time.sleep(0.5)

board.shutdown()


In [ ]:
import time
from pyfirmata import Arduino, util

# Connect to your Arduino (update port if needed)
board = Arduino('/dev/tty.usbmodem1101')  # macOS port

# Setup digital pins
LED_BLUE = board.get_pin('d:11:o')
LED_GREEN = board.get_pin('d:12:o')
LED_RED = board.get_pin('d:7:o')
SERVO = board.get_pin('d:5:s')     # Servo pin
BUTTON = board.get_pin('d:10:i')   # Button input
# No buzzer since you're skipping it

# Allow time for Firmata to initialize
time.sleep(2)

# Optional: Start iterator for reading inputs
it = util.Iterator(board)
it.start()
BUTTON.enable_reporting()

# Blink green LED and move servo
LED_GREEN.write(1)
SERVO.write(180)  # Open "door"
time.sleep(1)
SERVO.write(0)    # Close "door"
LED_GREEN.write(0)

# Wait for button press to reset
print("Waiting for button press...")
while True:
    if BUTTON.read() is False:  # Pull-up input (normally True, False when pressed)
        print("Button pressed. Resetting...")
        LED_RED.write(0)
        LED_BLUE.write(1)
        break
    LED_RED.write(1)
    time.sleep(0.1)

board.exit()


In [ ]:
import cv2
import time
import pickle
import numpy as np
from PIL import Image
from matplotlib import pyplot as plt
from pyfirmata import Arduino, util

# === Arduino Setup ===
board = Arduino('/dev/tty.usbmodem1101')  # Replace with your port

# Pin Assignments
LED_BLUE = board.get_pin('d:11:o')
LED_GREEN = board.get_pin('d:12:o')
LED_RED = board.get_pin('d:7:o')
SERVO = board.get_pin('d:5:s')
BUTTON = board.get_pin('d:10:i')  # pull-up logic (pressed = False)

# Start input iterator
it = util.Iterator(board)
it.start()
BUTTON.enable_reporting()

# Initialize pin states
LED_BLUE.write(1)   # idle state
LED_GREEN.write(0)
LED_RED.write(0)

time.sleep(2)  # Give Arduino time to stabilize

# === Webcam Capture ===
camera = cv2.VideoCapture(0)

# Capture and allow exposure to stabilize
for _ in range(3):
    ret, img_captured = camera.read()
    time.sleep(0.3)

cv2.imwrite("capturedPicture.png", img_captured)
camera.release()

# === Face Recognition Setup ===
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
recognizer = cv2.face.LBPHFaceRecognizer_create()
recognizer.read("trainer.yml")

# Load label names
with open("labels.pickle", "rb") as f:
    original_labels = pickle.load(f)
    labels = {v: k for k, v in original_labels.items()}

# === Read Captured Image ===
image = cv2.imread("capturedPicture.png")
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Detect faces
faces = face_cascade.detectMultiScale(
    gray,
    scaleFactor=1.21,
    minNeighbors=4,
    minSize=(20, 20),
    flags=cv2.CASCADE_SCALE_IMAGE
)

print(f"Faces found: {len(faces)}")

access_granted = False
confidence = 0

for (x, y, w, h) in faces:
    cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 0), 2)
    roi_gray = gray[y:y+h, x:x+w]
    id_now, confidence = recognizer.predict(roi_gray)
    print("Predicted Label:", labels.get(id_now, "Unknown"))
    print("Confidence:", confidence)
    if confidence >= 70:
        access_granted = True
        break

# === Visual Feedback ===
plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
plt.title("Face Recognition Result")
plt.axis("off")
plt.show()

# === Control Logic ===

if access_granted and len(faces) > 0:
    print("[✅] Face recognized. Access granted.")
    LED_BLUE.write(0)
    LED_GREEN.write(1)
    SERVO.write(180)      # Open "door"
    time.sleep(1)
    SERVO.write(0)        # Close "door"
    LED_GREEN.write(0)
    LED_BLUE.write(1)
else:
    print("[❌] Access denied. Security alert.")
    LED_BLUE.write(0)
    LED_RED.write(1)

    print("Waiting for reset button...")
    while True:
        if BUTTON.read() is False:  # Button pressed (active low)
            print("System Reset.")
            LED_RED.write(0)
            LED_BLUE.write(1)
            break


In [ ]:
import numpy as np
import os
import cv2
from matplotlib import pyplot as plt
import pickle
import time
from pyfirmata import Arduino, util

DIGITAL_B = 11  # Blue LED
DIGITAL_G = 12  # Green LED
DIGITAL_R = 7   # Red LED
SERVO_PIN = 5
BUTTON_PIN = 10
WAIT_TIME = 2

flag = True

# === Setup Arduino ===
board = Arduino('/dev/tty.usbmodem1101')  # Replace with your port

# Start iterator for reading button input
it = util.Iterator(board)
it.start()
time.sleep(1)

# Setup pins
led_b = board.get_pin(f'd:{DIGITAL_B}:o')
led_g = board.get_pin(f'd:{DIGITAL_G}:o')
led_r = board.get_pin(f'd:{DIGITAL_R}:o')
servo = board.get_pin(f'd:{SERVO_PIN}:s')
button = board.get_pin(f'd:{BUTTON_PIN}:i')
button.enable_reporting()

# Initial LED state
led_b.write(1)


# === Take Picture with Webcam ===
camera = cv2.VideoCapture(0)
for i in range(3):
    success, img_captured = camera.read()
    time.sleep(0.3)
cv2.imwrite('capturedPicture.png', img_captured)
camera.release()

# === Load Models and Labels ===
faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
recognizer = cv2.face.LBPHFaceRecognizer_create()
recognizer.read("trainer.yml")

with open("labels.pickle", "rb") as pickleFile:
    original_labels = pickle.load(pickleFile)
    labels = {v: k for k, v in original_labels.items()}

# === Process Captured Image ===
imagePath = 'capturedPicture.png'
image = cv2.imread(imagePath)
image_RGB = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
plt.title("Original Image")
plt.imshow(image_RGB)
plt.xticks([]), plt.yticks([])
plt.show()

gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
plt.title("Gray Image")
plt.imshow(gray, cmap='gray')
plt.xticks([]), plt.yticks([])
plt.show()

faces = faceCascade.detectMultiScale(
    gray,
    scaleFactor=1.21,
    minNeighbors=4,
    minSize=(20, 20),
    flags=cv2.CASCADE_SCALE_IMAGE
)
print(f'Faces found: {len(faces)}')

# === Face Recognition ===
confidence = 0
for (x, y, w, h) in faces:
    cv2.rectangle(image_RGB, (x, y), (x+w, y+h), (0, 255, 0), 5)
    roi_gray = gray[y:y+h, x:x+w]
    id_now, confidence = recognizer.predict(roi_gray)
    print("confidence =", confidence)
    # print("Face Recognized")
    # print("Access Granted")

plt.title("Faces Found")
plt.imshow(image_RGB)
plt.xticks([]), plt.yticks([])
plt.show()

# === Access Control Logic ===
if confidence >= 70 and len(faces) > 0:
    led_b.write(0)
    led_g.write(1)
    servo.write(180)
    time.sleep(1)
    servo.write(0)
    led_g.write(0)
    led_b.write(1)
    print("Access Granted")
    print("Opening the secret exit")
else:
    print("Access Denied")
    print("Security Breach")
    led_b.write(0)
    while flag:
        led_r.write(1)
        time.sleep(0.5)
        led_r.write(0)
        time.sleep(0.5)
        btn_value = True
        if btn_value is False:  # button pressed
            print("System Reset")
            led_b.write(1)
            flag = False
            break

board.exit()
